# 🧮 Laboratorio Interactivo: Support Vector Machines

[![Open in Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/boiro9/OptimizacionIA/main?filepath=Laboratorio_SVM_APP.ipynb)

En esta práctica vamos a ilustrar en la práctica el funcionamiento de la técnica de **Support Vector Machines (SVM)** como técnica de clasificación de una variable respuesta con dos etiquetas.


# 🚀  Instalación de paquetes:

Los requerimientos para poder ejecutar este jupyter son los siguientes:

In [1]:
# Instalar ipywidgets para que funcionen los widgets dentro de jupyter:
!pip install ipywidgets
# Instalar numpy, plotly, pandas, pyomo, amplpy
!pip install numpy plotly pandas pyomo amplpy 
# Instalar los solvers: HiGHS, CBC, Couenne, Bonmin, Ipopt, SCIP y GCG
!python -m amplpy.modules install coin highs scip gcg -q

In [2]:
import pyomo.environ as pe
from amplpy import modules

# SVM with Pyomo
def solve_SVM(Ydata,Xdata):
    
    m = pe.ConcreteModel()
    
    nvars = len(Xdata.columns)
    nobs  = len(Ydata)

    # Sets
    m.I = pe.RangeSet(0,nobs-1)
    m.J = pe.RangeSet(0,nvars-1)
    
    # Creamos las variables del problema: w y b
    m.w = pe.Var(m.J)
    m.b = pe.Var()

    # Funcion objetivo
    @m.Objective(sense=pe.minimize)
    def Obj(m):
        return (1/2)*sum(m.w[j]**2 for j in m.J)

    # Restricción de separacion de clases
    @m.Constraint(m.I)
    def cons_clases_rule(m, i): #TODO: FALTA ENUMERATE PARA SABER VARIABLE
        return Ydata[i]*(sum(m.w[j]*Xdata.iloc[i,j] for j in m.J)+m.b)-1>= 0    

    # Configuramos el solver:
    solver_name = "ipopt"  # Opciones: "highs", "cbc", "couenne", "bonmin", "ipopt", "scip", "gcg".
    solver_ipopt = pe.SolverFactory(
        solver_name+"nl",
        executable=modules.find(solver_name),
        solve_io="nl"
    )
    # Resolvemos
    results = solver_ipopt.solve(m)

    # Status solver
    print(f"*********************************")
    print(f"  Status: {results.solver.status}")
    print(f"  Termination: {results.solver.termination_condition}")
    print(f"*********************************")
   
    return m

# Margin SVM with Pyomo
def solve_suave_SVM(Ydata,Xdata, M=10):
    
    m = pe.ConcreteModel()
    
    nvars = len(Xdata.columns)
    nobs  = len(Ydata)

    # Sets
    m.I = pe.RangeSet(0,nobs-1)
    m.J = pe.RangeSet(0,nvars-1)
    
    # Creamos las variables del problema: w y b
    m.w = pe.Var(m.J)
    m.b = pe.Var()
    m.s = pe.Var(m.I,domain=pe.NonNegativeReals)

    # Funcion objetivo
    @m.Objective(sense=pe.minimize)
    def Obj(m):
        return (1/2)*sum(m.w[j]**2 for j in m.J)+M/(nobs)*sum(m.s[i] for i in m.I)

    # Restricción de separacion de clases
    @m.Constraint(m.I)
    def cons_clases_rule(m, i): #TODO: FALTA ENUMERATE PARA SABER VARIABLE
        return Ydata[i]*(sum(m.w[j]*Xdata.iloc[i,j] for j in m.J)+m.b)-1>= -m.s[i]    

    # Configuramos el solver:
    solver_name = "ipopt"  # Opciones: "highs", "cbc", "couenne", "bonmin", "ipopt", "scip", "gcg".
    solver_ipopt = pe.SolverFactory(
        solver_name+"nl",
        executable=modules.find(solver_name),
        solve_io="nl"
    )
    # Resolvemos
    results = solver_ipopt.solve(m)

    # Status solver
    print(f"*********************************")
    print(f"  Status: {results.solver.status}")
    print(f"  Termination: {results.solver.termination_condition}")
    print(f"*********************************")
   
    return m

In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def ejecutar_SVM(data_name,pyomo_model,Mvalue):

    # Lectura datos:
    datos = pd.read_csv(data_name, delimiter=',')
    display(datos)

    # Preparamos los datos de entrada del problema:
    Ydata = datos['y']          # Variable respuesta         
    Xdata = datos[['x1', 'x2']] # Variables explicativas

    #################
    # Representación gráfica
    #################
    fig = go.Figure()
    
    # Añadir puntos
    fig.add_trace(go.Scatter(
        x=Xdata['x1'], y=Xdata['x2'],
        mode='markers',
        marker=dict(
            color=Ydata,
            colorscale=[[0, 'red'], [1, 'blue']], # Asignar -1 a rojo, 1 a azul
            line=dict(width=1, color='DarkSlateGrey')
        ),
        name='Datos'
    ))
    
    fig.update_layout(title="Datos:", xaxis_title="x1", yaxis_title="x2")
    fig.show()

    if pyomo_model=="SVM":
        modelo_svm = solve_SVM(Ydata,Xdata)
    elif pyomo_model=="SVM_suave":
        modelo_svm = solve_suave_SVM(Ydata,Xdata, M=Mvalue)

    # Resultados:
    w=[modelo_svm.w[0](),modelo_svm.w[1]()]
    b=modelo_svm.b()
    print('Funcion Objetivo: ',modelo_svm.Obj())
    print('Vectores Soporte: ', w)
    print('Término Independiente: ', b)

    ##################
    # Añadimos hiperplano separador, margen positivo, margen negativo y vectores soporte
    ##################
    # Dominio de x1
    x_plot = np.array([np.min(Xdata['x1']) - 1, np.max(Xdata['x1']) + 1])
    
    # w1*x1 + w2*x2 + b = 0 => x2 = (-w1*x1-b)/w2
    y1 = (-w[0] * x_plot - b) / (w[1] if abs(w[1])>1e-8 else (w[1]+1e-8))
    # w1*x1 + w2*x2 + b = 1 => x2 = (-w1*x1-b-1)/w2
    y_pos = (-w[0] * x_plot - b - 1) / (w[1] if abs(w[1])>1e-8 else (w[1]+1e-8))
    # w1*x1 + w2*x2 + b = -1 => x2 = (-w1*x1-b+1)/w2
    y_neg = (-w[0] * x_plot - b + 1) / (w[1] if abs(w[1])>1e-8 else (w[1]+1e-8))
    
    fig.add_trace(go.Scatter(x=x_plot, y=y1,
                        mode='lines',
                        name='Hiperplano SVM (w·x+b=0)'))
    
    fig.add_trace(go.Scatter(x=x_plot, y=y_pos,
                        mode='lines',
                        name='Margen +1 (w·x+b=1)'))
    fig.add_trace(go.Scatter(x=x_plot, y=y_neg,
                        mode='lines',
                        name='Margen -1 (w·x+b=-1)'))

    # Vectores soporte:
    dist = Ydata * (Xdata.dot(w) + b)
    support_mask = np.isclose(dist, 1.0, atol=1e-5)
    sv = Xdata[support_mask]
    fig.add_trace(go.Scatter(x=sv.iloc[:,0], y=sv.iloc[:,1], mode='markers', marker=dict(size=14, line=dict(color='purple', width=3), color='rgba(0,0,0,0)'), name='Soporte'))
    fig.show()
    


In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class InteractiveSVM:
    def __init__(self):
        self.data_name = widgets.Dropdown(options=['svm_data.csv','svm_nonseparable_data.csv'], description='Datos')
        self.svm_model = widgets.Dropdown(options=['SVM','SVM_suave'], description='Modelo SVM')
        self.Mvalue    = widgets.FloatText(
                                value=10,
                                description='Penalización (M):',
                                disabled=False
                            )
        self.solve_button = widgets.Button(description='Resolver')
        self.out = widgets.Output()
        self.solve_button.on_click(self.on_solve)
    def on_solve(self, b):
        with self.out:
            clear_output()
            try:
                ejecutar_SVM(data_name=self.data_name.value,pyomo_model=self.svm_model.value,Mvalue=self.Mvalue.value)
            except Exception as e:
                print('Error al ejecutar:', e)
    def show(self):
        box = widgets.VBox([widgets.HBox([self.data_name, self.svm_model]),
                            widgets.HBox([ self.Mvalue]),
                            self.solve_button, self.out])
        display(box)

In [5]:
InteractiveSVM().show()